In [ ]:
]activate ..

In [ ]:
include("main.jl")

In [ ]:
using .AudioInference

In [ ]:
trr = tones_with_noise(10.); nothing

In [ ]:
vis_and_write_wave(trr, "trr.wav")

In [ ]:
AudioInference.savefig("truth.pdf")

In [ ]:
(initial_tr, weight) = generate_initial_tr(trr)
weight

In [ ]:
using PyPlot
using Dates

In [ ]:
function get_avg_likelihoods(initial_trs, run_inf!, iters)
  likelihoods = zeros(Float64, iters)
  starttime = Dates.now()
  for (i, initial_tr) in enumerate(initial_trs)
    print("Running trial $i...;")
    println(" $(Dates.now() - starttime) ms ellapsed in total")
    (l, record!) = AudioInference.get_worldmodel_likelihood_tracker_and_recorder()
    run_inf!(initial_tr, iters, record!)
    likelihoods += l
  end
  likelihoods /= length(initial_trs)
  return likelihoods
end

In [ ]:
#initial_trs = [AudioInference.simulate(AudioInference.generate_scene, AudioInference.args) for _=1:5]

In [ ]:
generic_likelihoods = get_avg_likelihoods(fill(initial_tr,5), AudioInference.do_generic_inference, 600)
plot(generic_likelihoods)

In [ ]:
bd_likelihoods = get_avg_likelihoods(fill(initial_tr,5), AudioInference.do_birth_death_inference, 600)
plot(bd_likelihoods)

In [ ]:
sm_likelihoods = get_avg_likelihoods(fill(initial_tr,5), AudioInference.do_split_merge_inference, 600)
plot(sm_likelihoods)

In [ ]:
bot, top = 1, 600
plot(bot:top, generic_likelihoods[bot:top], label="generic")
plot(bot:top, bd_likelihoods[bot:top], label="birth/death")
plot(bot:top, sm_likelihoods[bot:top], label="split/merge")
xlabel("number of inference iterations")
ylabel("log likelihood of observed sound given inferred waves")
title("Quality of inferred waveforms over time")
legend()

In [ ]:
function get_avg_likelihoods_and_counts(initial_trs, run_inf!, iters)
  likelihoods = zeros(Float64, iters)
  counts = []
  starttime = Dates.now()
  for (i, initial_tr) in enumerate(initial_trs)
    print("Running trial $i...;")
    println(" $(Dates.now() - starttime) ms ellapsed in total")
    (l, record!) = AudioInference.get_worldmodel_likelihood_tracker_and_recorder()
    tr = run_inf!(initial_tr, iters, record!)
    push!(counts, tr[:kernel => :n_tones])
    likelihoods += l
  end
  likelihoods /= length(initial_trs)
  return (likelihoods, counts)
end

In [ ]:
(l, c) = get_avg_likelihoods_and_counts(fill(initial_tr,2), AudioInference.do_birth_death_inference, 600)
c

In [ ]:
(l, c) = get_avg_likelihoods_and_counts(fill(initial_tr,2), AudioInference.do_split_merge_inference, 600)
c

In [ ]:
using Pkg; Pkg.add("ProfileView")

In [ ]:
using Profile; using ProfileView;

In [ ]:
@profile get_avg_likelihoods_and_counts(fill(initial_tr,5), AudioInference.do_split_merge_inference, 600)

In [ ]:
ProfileView.view()

In [ ]:
function plot_gtg(gtg, duration, audio_sr, vmin, vmax;colors="Blues",plot_colorbar=false)
    
    max_freq=audio_sr/2
    imshow(gtg, cmap=colors, origin="lower", extent=(0, duration, 0, max_freq),vmin=vmin, vmax=vmax, aspect=1/1300)
    locs, labels = yticks();
    lowlim = AudioInference.freq_to_ERB(1.)
    hilim = AudioInference.freq_to_ERB(max_freq)
    fs = Int.(floor.(AudioInference.ERB_to_freq(range(lowlim, stop=hilim, length=length(locs)))))
    setp(gca().set_yticklabels(fs), fontsize="small")
    if plot_colorbar
        plt.colorbar()
    end
end

In [ ]:
function vis(tr)
    duration, _, sr, = AudioInference.get_args(tr)
    gram, scene_wave, = AudioInference.get_retval(tr)
  #  wavwrite(scene_wave/maximum(abs.(scene_wave)), title, Fs=sr)
 # display(gram)
    plot_gtg(gram, duration, sr, 0, 100)
end

In [ ]:
trr = tones_with_noise(10.); nothing

In [ ]:
vis(trr)

In [ ]:
tr = AudioInference.do_birth_death_inference(initial_tr, 500, (tr,) -> ())

In [ ]:
vis(tr)